# Solve with Algorithm2

To include Line numbers: select cell -> ctrl + m -> l (= ell) (without ctrl)

### Disitrubtion dependence 
The code as written below works for *all* instances. 

However for possible speed-up we've added minor changes for instances from the discrete Uniform distribution. Our changes can be found as comment in the code. These changes need not be made in order for the code to work on instances form the Uniform distribution. 


In [1]:
using JuMP
using CPLEX

# Algorithm 2


In [2]:
function Algorithm2(n::Int,d::Int,C::Array{Float32,3})
#function Algorithm2(n::Int,d::Int,m::Int,C::Array{Int64,3})
    
     mi = minimum(C);  # remove this line for Uniform specific
     ma = maximum(C);  # remove this line for Uniform specific
    
     OPT = ma+1;   # OPT = 2*m; 
     ArgOpt = [];  

# step 1 select two elements from the ground set; x_1 = (p1,p2) and x_2 = (q1,q2)
    for p1 in 1: n-1
        for p2 in 1:n 

            for q2 in 1:n
                if(q2 != p2)
                    for q1 in p1+1:n
                
                        Sol::Float32 = ma+1;    # Sol::Float32 = 2*m;
                        delta::Float32 = ma+1;    # delta::Float32 = 2*m;
                        upper = Float32[ma+1 for k in 1:d]; # upper = Float32[2*m for k in 1:d]
                        lower = Float32[mi-1 for k in 1:d]; # lower = Float32[0 for k in 1:d];
                        index = [];  #used for finding arg_k
                        TestNeeded = 1; # check whether or not model needs to be made and solved. 

# step 2: calculate \Delta
                        for k in 1:d
                            if(C[p1,p2,k] <= C[q1,q2,k])
                                lower[k] = C[p1,p2,k]
                                upper[k] = C[q1,q2,k]
                            else
                                upper[k] = C[p1,p2,k]
                                lower[k] = C[q1,q2,k]
                            end
                        end

                        delta = maximum(upper-lower);   
                        
                        if(delta <= OPT)
                            
# step 3: Y := X
                             O = Bool[1 for i in 1:n, j in 1:n]; #  matrix for max weight Assignment Problem
                            
# step 4-6: remove elements if needed; i.e. set weight in matrix O to 0      
                            for b in 1:n, a in 1:n 
                                if (O[a,b] == 1)
                                    if(maximum(abs(C[a,b,k] - C[p1,p2,k]) for k in 1:d) > delta || maximum(abs(C[a,b,k] - C[q1,q2,k]) for k in 1:d)  > delta)
                                          O[a,b] = 0     
                                      end
                                end
                            end # for a,b
                            
                # If there exists any row, or column in which no 1 occurs; then we know it is infeasible
                # no model needs to be made and solved. 
                            for a in 1:n 
                                if( sum( O[a,x] for x in 1:n) < 0.9)
                                    TestNeeded = 0
                                end
                            end
                            
                            if(TestNeeded > 0.1)
                                for b in 1:n 
                                    if( sum( O[x,b] for x in 1:n) < 0.9)
                                        TestNeeded = 0
                                    end
                                end
                            end

# step 7: feasibility oracle; maximum weight assignment 
                            
                            if(TestNeeded > 0.1)
                                
                               modelFeas = Model(solver=CplexSolver(CPX_PARAM_SCRIND=0));  
                                @variable(modelFeas,0 <= y[1:n,1:n] <= 1, Int);

                                @constraints(modelFeas, begin       
                                    Assignment1[x1 in 1:n], sum(y[x1,x2] for x2 = 1:n) == 1
                                    Assignment2[x2 in 1:n], sum(y[x1,x2] for x1 = 1:n) == 1
                                end);

                                @objective(modelFeas, Max, sum(y[x1,x2]*O[x1,x2] for x2 = 1:n, x1 in 1:n));
                                status = solve(modelFeas);
                                getObj = getobjectivevalue(modelFeas); 

# step 8 if feasible, then sompute Sol. 
                                if(getObj > n - 0.5) # allow for numerical inaccuracy
                                    
                                    for(z2 in 1:n)
                                        for(z1 in 1:n)
                                            if(getvalue(y)[z1,z2] == 1)
                                                for k in 1:d
                                                    if(C[z1,z2,k] <= lower[k])
                                                        lower[k] = C[z1,z2,k]
                                                    elseif(C[z1,z2,k] >= upper[k])
                                                        upper[k] = C[z1,z2,k]
                                                    end
                                                end # end for k
                                            end
                                        end # for z1
                                    end #for z2

                                    Sol = maximum(upper-lower);
                                    index = find(a->a==Sol, upper-lower);

# step 9; keep best solution value found so far
                                    
                                    if(OPT > Sol)
                                         ArgOpt = index;
                                        OPT = Sol;
                                    end

                                    end # if getObj
                                end # if TestNeeded
                            end # if delta <= OPT
    
                        
                        
                        end # if q2 != p2
                    end # for q2
                end# for q1
            end # for p2
        end # end for p1
    
    print(" ",OPT, " ", sum(ArgOpt)/length(ArgOpt)," ", length(ArgOpt), " "); 
end

Algorithm2 (generic function with 1 method)

# choose instance type + run

First call of the function requires more running time, due to compiling.
Hence we first solve a small test instance, before running the actual instances.

In [ ]:
   println("testing + compiling")
    C3 = Float32[1 for i in 1:3, j in 1:3, k in 1:3];
    temp = readdlm(pwd()"/test/Norm1.txt")
    for(j in 1:3)
        for i in 1:3
          for k in 1:3
             C3[i,j,k] = temp[3*(i-1)+j,k];    
          end
        end
    end
    @time Algorithm2(3,3,C3)


#############  Run all instances Class 1  #########

for m in [100,1000]
    for n in [10,20]  
        println(" ")
        println(" ")
        for d in [2,100,300]
            println("ALG2:Uniform", m, "n=", n, "d=", d)
            for iter in 1:10
                #local C2 = Int64[1 for i in 1:n, j in 1:n, k in 1:d];
                local C2 = Float32[1 for i in 1:n, j in 1:n, k in 1:d];
                temp = readdlm(pwd()"/U(0,"*string(m)*")/n"*string(n)*"d"*string(d)*"/Ist"*string(iter)*".txt")    
                for( j in 1:n)
                    for i in 1:n
                        for k in 1:d
                            C2[i,j,k] = temp[n*(i-1)+j,k];    
                        end
                    end
                end
                print("Ist", iter)
               #@time Algorithm2(n,d,m,C2)
                @time Algorithm2(n,d,C2)
            end
        end
    end
end

for E in [50,500]
    for S in [1,10]
        println(" ")
        println(" ")
        for n in [10,20]
            for d in [2,100,300]
                println("ALG2:Normal(",E,",",S,")n=", n, "d=", d)
                for iter in 7:10

                    local C2 = Float32[1 for i in 1:n, j in 1:n, k in 1:d];
                    temp = readdlm(pwd()"/N("*string(E)*","*string(S)*")/n"*string(n)*"d"*string(d)*"/Ist"*string(iter)*".txt")
                    for( j in 1:n)
                        for i in 1:n
                            for k in 1:d
                                C2[i,j,k] = temp[n*(i-1)+j,k];    
                            end
                        end
                    end
                    print("Ist", iter)
                    @time Algorithm2(n,d,C2)

                end
            end
        end 
    end
end


testing + compiling
 1.1839523 3.0 1   4.905891 seconds (5.30 M allocations: 221.216 MB, 1.65% gc time)
 
 
ALG2:Uniform100n=10d=2
Ist1 35.0 1.5 2   0.201699 seconds (2.59 M allocations: 53.618 MB, 6.77% gc time)
Ist2 34.0 2.0 1   0.290590 seconds (2.14 M allocations: 48.086 MB, 1.51% gc time)
Ist3 40.0 1.0 1   0.630266 seconds (3.29 M allocations: 82.129 MB, 1.35% gc time)
Ist4 46.0 1.0 1   1.075970 seconds (3.91 M allocations: 102.683 MB, 1.27% gc time)
Ist5 42.0 1.0 1   0.711345 seconds (3.40 M allocations: 84.146 MB, 1.46% gc time)
Ist6 35.0 2.0 1   0.356743 seconds (2.75 M allocations: 61.028 MB, 1.78% gc time)
Ist7 38.0 2.0 1   0.507249 seconds (2.81 M allocations: 63.705 MB, 1.23% gc time)
Ist8 38.0 2.0 1   0.572903 seconds (2.88 M allocations: 71.583 MB, 1.13% gc time)
Ist9 38.0 1.0 1   0.558413 seconds (3.16 M allocations: 76.193 MB, 1.49% gc time)
Ist10